<a href="https://colab.research.google.com/github/maxigaarp/Gestion-de-datos-2022/blob/main/Taller_4_Gesti%C3%B3n_de_datos_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(stringr)
install.packages("RSQLite")
library(RSQLite)
library(data.table)

# Vectorizacion

Hay operaciones que, si las aplicamos a un vector, son aplicadas a todos sus elementos.



In [ ]:
mi_vector <- 1:10

mi_vector

In [ ]:
mi_vector ^ 2

Esto es mucho mas facil y rapido que a aplicar el exponente a cada elemento del vector. 

Como todo lo que ocurre en R es una función, podemos decir que al vectorizar estamos aplicando una función a cada elemento de un vector. La familia de funciones apply nos permite implementar esto en estructuras de datos distintas a los vectores.

## Funciones


Recordemos que en R podemos definir funciones, es decir
area_cuad <- function(lado1, lado2) {
  lado1 * lado2
}

In [ ]:
es_primo <-function(numero){
  if(numero<2){
    return(FALSE)
  }
  if(numero == 2){
    return(TRUE)
  }
  for(i in 2:(numero-1)){
    if(numero%%i == 0){
      return(FALSE)
    }
  }
  return(TRUE)
}

print(es_primo(123121))
print(es_primo(123121231))

In [ ]:
es_primo(2)

Sin embargo, si queremos aplicar esta función a un vector o serie de elementos, empezamos a observar problemas

In [ ]:
print(es_primo(mi_vector))

In [ ]:
for(c in mi_vector){
  print(es_primo(c))
}

El problema de esta solución es adaptarlo a data frames (consumo de memoria) y tiempo de ejecución. 

## Apply
Existen muchas funciones de la familia apply y todas tienen alguna aplicacion especifica, 
 * apply()
 * eapply()
 * lapply()
 * mapply() 
 * rapply()
 * sapply()
 * tapply()
 * vapply()

Como caracteristica general, todas las funciones de esta familia tienen una característica en común: reciben como argumentos a un objeto y al menos una función.

Veremos las principales que son apply y lapply


Apply: es la función central de esta familia, lo que hace es aplicar funciones **vectoriales** sobre matrices (o data frames). 

Esta función se aplica sobre la matriz de la siguiente forma:

apply(MATRIX, MARGIN, FUN)

MATRIX en la que uno quiere aplicar la función, por ejemplo nuestra BBDD

MARGIN es la dimensión en que uno quiere aplicar la función. Si el valor es 1 la función se aplica en cada fila, si el valor es 2, se aplica en cada columna. si el valor es c(1,2) se aplica en cada elemento, mientras que si el valor es c(2,1) se devuelve la matriz transpuesta. En los dos últimos casos se retorna siempre una matriz, no una bbdd.

FUN es la función que uno determina.

Es especialmente importante conocer bien la función y entender que va  a retornar apply.

In [ ]:
apply(mi_vector, MARGIN = 1, FUN = es_primo)

In [ ]:
mi_matriz = matrix(mi_vector, nrow = 2)
mi_matriz
apply(mi_matriz, MARGIN = 2, FUN = es_primo)

In [ ]:
mi_matriz = matrix(mi_vector, nrow = 2)
mi_matriz
apply(mi_matriz, MARGIN = 2, FUN = sum)

In [ ]:
mi_matriz = matrix(mi_vector, nrow = 2)
mi_matriz
apply(mi_matriz, MARGIN = c(1,2), FUN = es_primo)

In [ ]:
mi_matriz = matrix(mi_vector, nrow = 2)
mi_matriz
apply(mi_matriz, MARGIN = c(2,1), FUN = es_primo)

Por otra parte, lapply es un caso especial de apply, diseñado para aplicar funciones a todos los elementos de una lista, retornando una lista.

lapply(LIST, FUN)

LIST es la lista que entregamos

FUN es nuestra función

Siempre retorna una lista



In [ ]:
lapply(mi_vector, es_primo)

Uno puede crear funciones sabiendo con que tipo de elementos va a trabajar, y apply() permite resolver los problemas de forma muy eficiente.

In [ ]:
suma_es_primo <-function(x){
  numero <- sum(x)
  if(numero<2){
    return(FALSE)
  }
  if(numero == 2){
    return(TRUE)
  }
  for(i in 2:(numero-1)){
    if(numero%%i == 0){
      return(FALSE)
    }
  }
  return(TRUE)
}


In [ ]:
mi_matriz = matrix(mi_vector, nrow = 5)
mi_matriz
apply(mi_matriz,MARGIN = 1, FUN = suma_es_primo)

# Ajustar Base de datos

Ir a https://www.cr2.cl/datos-de-precipitacion/?cp_Precipitacion=2

Descargar datos del 2019

In [ ]:
library(tidyverse)

In [ ]:
unzip("/content/cr2_prAmon_2019.zip")

In [ ]:
x <- read.csv("/content/cr2_prAmon_2019/cr2_prAmon_2019.txt", na = "-9999", header =F)


In [ ]:
x

In [ ]:
y <- setNames(as.data.frame(t(x[,-1])),as.character(x[,1]))

y[,1:20]

In [ ]:
y

In [ ]:
y10 <- y %>% select(c(1,6,7,tail(names(y), 12*10)))
y10


In [ ]:
plot(1:120,y["V878",tail(names(y), 12*10)])
lines(1:120,y["V878",tail(names(y), 12*10)])

In [ ]:
y10 %>%
  mutate(count = rowSums(is.na(y10)))%>%
  filter(count<5)


In [ ]:
plot(1:120,y["V6",tail(names(y), 12*10)])
lines(1:120,y["V6",tail(names(y), 12*10)])

In [ ]:
y10

In [ ]:
y10_pivot <- data.frame(apply(y10, 2, function(x) as.numeric(as.character(x)))) %>%
        pivot_longer(cols = X2010.01:X2019.12,
        names_to = c("Año", "Mes"),
        names_pattern = "X(....).(..)")

y10_pivot

In [ ]:
y10_pivot %>% filter(codigo_estacion==1201010)%>%
              group_by(Mes)%>%
              summarise(pp_aprox=mean(value, na.rm=T))%>%
              ggplot() + geom_line(aes(x=Mes, y=pp_aprox, group = 1))



Actividad:

 * Elija una posicion geografica que sea de su interes (para eso vaya a google maps busque un lugar y luego le extrae la latitud y longitud). Busque las 5 estaciones más cercanas a ese punto y calcule la interpolacion (inversa) en el punto para los 12 años de historia.

# Cargar datos a SQL

In [ ]:
system("gdown --id 1-q1ydcu6afA3LQ9uxlh9J9B9kvEJvrRs")
unzip("/content/DatosProyecto.zip")

In [ ]:
conn <- dbConnect(RSQLite::SQLite(), "mineduc.db")

In [ ]:
lista <- list.files("/content/content/drive/MyDrive/Gestion de Datos/Datos Proyecto/Datos asistencia")
directorio <- "/content/content/drive/MyDrive/Gestion de Datos/Datos Proyecto/Datos asistencia/"

for (name in lista){
  if (str_detect(name, ".rar")){
    foldername<-substr(name, 1, nchar(name)-4)
    system(paste("mkdir 'Asistencia/",foldername,"'",sep=""))
    system(paste("unrar x '",directorio,name,"' 'Asistencia/",substr(name, 1, nchar(name)-4),"/'", sep=""),intern = T)
  }
  else {
    unzip(paste(directorio,name,sep=""), exdir="Asistencia")
  }
}


In [ ]:
lista_de_csvs <- list.files("Asistencia",pattern = ".(CSV|csv)$", recursive = TRUE)

In [ ]:
lista_de_csvs

In [ ]:
aux1 <- NULL
cont <- 1
cod_ense1_ense2 <- fread(paste0("/content/Asistencia/",lista_de_csvs[1]))%>% select(COD_ENSE,COD_ENSE2)%>% unique

In [ ]:

for (i in lista_de_csvs) {
  if (grepl("_2015_",i)){
 aux1 =
fread(paste0("/content/Asistencia/",i)) %>%
left_join(cod_ense1_ense2, by = "COD_ENSE") %>%
select(AGNO, MES_ESCOLAR, RBD, RURAL_RBD, COD_DEPE, COD_DEPE2, COD_ENSE2, COD_ENSE, COD_GRADO, MRUN,
GEN_ALU, FEC_NAC_ALU, DIAS_ASISTIDOS, DIAS_TRABAJADOS, ASIS_PROMEDIO)%>% mutate(EDAD = AGNO-(FEC_NAC_ALU%/%100))

  apnd=if (i==lista_de_csvs[1]) FALSE else TRUE

  dbWriteTable(conn , name = "Asistencia_MRUN_RBD", 
                  value = aux1,
                  row.names = FALSE, header = !apnd, sep=',',append=apnd,
                  colClasses='character')
                  aux1 = NULL
  }
   if (!grepl("_2015_",i)){
 aux1 =
fread(paste0("/content/Asistencia/",i)) %>%
select(AGNO, MES_ESCOLAR, RBD, RURAL_RBD, COD_DEPE, COD_DEPE2, COD_ENSE2, COD_ENSE, COD_GRADO, MRUN,
GEN_ALU, FEC_NAC_ALU, DIAS_ASISTIDOS, DIAS_TRABAJADOS, ASIS_PROMEDIO)%>% mutate(EDAD = AGNO-(FEC_NAC_ALU%/%100))

apnd=if (i==lista_de_csvs[1]) FALSE else TRUE

  dbWriteTable(conn , name = "Asistencia_MRUN_RBD", 
                  value = aux1,
                  row.names = FALSE, header = !apnd, sep=',',append=apnd,
                  colClasses='character')
                  aux1 = NULL

  }

  print(2)


}

In [ ]:
dbGetQuery(conn, 'SELECT * FROM Asistencia_MRUN_RBD LIMIT 5')

In [ ]:
dbGetQuery(conn, 'SELECT count(MRUN) FROM Asistencia_MRUN_RBD 
                  Group by AGNO')